In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

IMG_SIZE = 224

setattr(tfds.image_classification.cats_vs_dogs,
        '_URL',
        "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")
(train, validation, test), metadata = tfds.load(
    'cats_vs_dogs',
    split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
    as_supervised=True,
    with_info=True,
)


get_label_name = metadata.features['label'].int2str

n = 10
for image, label in train.take(n):
  plt.imshow(image)
  plt.show()


def augmentImages(image,label):
  image = tf.cast(image,tf.float32)
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  image = tf.image.random_flip_left_right(image)
  image = tf.image.flip_left_right(image)
  label = tf.one_hot(label,1)
  return tf.cast(image,tf.float32)/255.0, label

def resize_val(image,label):
  image = tf.image.resize(image,(IMG_SIZE,IMG_SIZE))
  label = tf.one_hot(label,1)
  return tf.cast(image,tf.float32)/255.0,label

def make_pred(path,model):
  img = Image.open(path)
  img = np.asarray(img)
  img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
  img = img / 255.0
  img = np.expand_dims(img, axis=0)
  if model.predict(img)[0][0] >= 0.5:
    return 'cat'
  else:
    return 'dog'

train_ds = train.map(augmentImages)
train_batches = train_ds.shuffle(len(train_ds)).batch(32)
val_ds = validation.map(resize_val)
val_batches = val_ds.shuffle(len(val_ds)).batch(32)


model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32,3,activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
  tf.keras.layers.MaxPooling2D(2),
  tf.keras.layers.Conv2D(64,3,activation='relu'),
  tf.keras.layers.MaxPooling2D(2),
  tf.keras.layers.Conv2D(128,3,activation='relu'),
  tf.keras.layers.MaxPooling2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64,activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1,activation='sigmoid')
])

# model.compile(optimizer = tf.keras.optimizers.Adam(1e-4),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
# model.fit(train_batches,
#           epochs=20,
#           validation_data=val_batches,
#           validation_steps=len(val_batches))
# model.evaluate(train_batches)
test_ds =test.map(resize_val).batch(8)
#loaded_model = tf.keras.models.load_model('dogs_vs_cats.h5')
#loaded_model.evaluate(test_ds)
#make_pred(r"C:\Users\mturri\OneDrive - BUSINESS INTEGRATION PARTNERS SPA\Desktop\istinto-predatorio-cane-1.jpeg",model)

mn = tf.keras.applications.mobilenet.MobileNet()
mn.summary()
tl_model =tf.keras.models.Sequential()
for layer in mn.layers[:-1]:
  tl_model.add(layer)

tl_model.add(tf.keras.layers.Dense(128,activation='relu'))
tl_model.add(tf.keras.layers.Dense(1,activation='softmax'))

tl_model.summary()

tl_model.compile(optimizer = tf.keras.optimizers.Adam(1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# history = tl_model.fit(train_batches,
#           epochs=5,
#           validation_data=val_batches,
#           validation_steps=len(val_batches))

tl_model = tf.keras.models.load_model('mobile_net.h5')
tl_model.evaluate(train_batches)
test_ds =test.map(resize_val).batch(8)
tl_model.evaluate(test_ds)


def plot_learning_curves(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  plt.plot(acc)
  plt.plot(val_acc)
  plt.title('Accuracy')
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.show()

  plt.plot(loss)
  plt.plot(val_loss)
  plt.title('Loss')
  plt.xlabel('epochs')
  plt.ylabel('loss')
  plt.show()


#plot_learning_curves(history)
print(f"This is a {make_pred('dog.jpg',tl_model)} image")